In [6]:
using PlotlyJS
using Statistics
using JLD2
using BenchmarkTools

WebIO._IJuliaInit()

In [8]:
include("../src/sat.jl")
include("../../2/src/file_loader.jl")
tst = "/home/sutymate/School/KOP/HW/5/data/test/simple5-2"
A = "/home/sutymate/School/KOP/HW/5/data/wuf-A1/wuf20-88-A1"
A1 = "/home/sutymate/School/KOP/HW/5/data/wuf-A1/wuf20-88-A1/wuf20-014-A.mwcnf"
Q = "/home/sutymate/School/KOP/HW/5/data/wuf-Q1/wuf20-78-Q1"
Q1 = "/home/sutymate/School/KOP/HW/5/data/wuf-Q1/wuf20-78-Q1/wuf20-01.mwcnf"
instances = readFileSat(A1)

ax = []
T = [100, 200]
TF = [10]
I = [50]
C = [0.99]#, 0.999, 0.9995]#, 0.995]
for instance in instances
    for t in T
        for tf in TF
            for i in I
                for c in C
                    profit, absolute, y, y_best, steps = sa(instance, t, tf)
                    desc = string(t)*", "*string(tf)*", "*string(i)*", "*string(c)
                    push!(ax, scatter(y=y, mode="lines", name=desc))
                    push!(ax, scatter(y=y_best, mode="markers"))
                    println(profit, " ", desc, " ", steps)
                end
            end
        end
    end
end


0.5802411873840445 100, 10, 50, 0.99 11500
0.45918367346938777 200, 10, 50, 0.99 14950


In [17]:
instances = readFileSat(A1)
ax=[]
profit, absolute, y, y_best, steps = sa(instances[1], 200, 0.01)
# desc = string(t)*", "*string(tf)*", "*string(i)*", "*string(c)
push!(ax, scatter(y=y, mode="lines"))#, name=desc))
# push!(ax, scatter(y=y_best, mode="markers"))
println(profit, absolute)
p = plot([p for p in ax], Layout(title="optimal solution ", showlegend=true))

0.903614457831325375


data: [
  "scatter with fields mode, type, and y"
]

layout: "layout with fields margin, showlegend, template, and title"

In [ ]:
save_object("bench100-BB.jld2")

In [186]:
rel_errorSA = []
rel_errorREDUX = []
for b in benchSA
    push!(rel_errorSA, mean(b[2]))
end
for r in benchREDUX
    push!(rel_errorREDUX, r[2])
end

println(mean(rel_errorSA))
println(std(rel_errorSA))
println()
println(mean(rel_errorREDUX))
println(std(rel_errorREDUX))

0.010739497004540416
0.004264482442523088

0.019592656359368982
0.01290043513580828


In [184]:
include("old_simulated.jl")
ax = []
filename = "../data/40.dat"
instances = readFile(filename)

t = 1000
tf = 10
i = 50
c = 0.999
attempts = 3
cnt = 1
benchPD = Dict()
benchREDUX = Dict()
benchSA = Dict()
TIMEbenchPD = 0
TIMEbenchREDUX = 0
TIMEbenchSA = 0
for instance in instances[1:10]
    pd = price_decompose(instance[1], instance[2])[1]
    redux = redux_greedy(instance[1], instance[2])
    benchREDUX[cnt] = abs(pd - redux) / pd
    
    benchPD[cnt] = pd

    benchSA[cnt] = []
    tsa = 0
    tredux = 0
    tpd = 0
    for a in 1:attempts
        tsa += @elapsed sax, best, y, y_best, steps = sa(instance, t, tf, i, c)
        tredux += @elapsed redux = redux_greedy(instance[1], instance[2])
        tpd += @elapsed pd = price_decompose(instance[1], instance[2])[1]
        println(sax, " ", redux, " ", " ", pd, " ", a, " cnt ", cnt)#, desc, " ", steps)
        push!(benchSA[cnt], (abs(pd - sax) / pd ))
    end
    TIMEbenchSA += tsa/attempts
    TIMEbenchREDUX += tredux/attempts
    TIMEbenchPD += tpd/attempts
    cnt += 1
end

32161 31589  32413 1 cnt 1
31910 31589  32413 2 cnt 1
32113 31589  32413 3 cnt 1
34681 34426  35056 1 cnt 2
34731 34426  35056 2 cnt 2
34761 34426  35056 3 cnt 2
39010 39328  39509 1 cnt 3
39006 39328  39509 2 cnt 3
38990 39328  39509 3 cnt 3
34908 34102  35180 1 cnt 4
34883 34102  35180 2 cnt 4
34919 34102  35180 3 cnt 4
37707 37368  37998 1 cnt 5
37694 37368  37998 2 cnt 5
37729 37368  37998 3 cnt 5
37580 35866  37601 1 cnt 6
37549 35866  37601 2 cnt 6
37339 35866  37601 3 cnt 6
38934 38510  39324 1 cnt 7
39030 38510  39324 2 cnt 7
38933 38510  39324 3 cnt 7
35597 36096  36096 1 cnt 8
35417 36096  36096 2 cnt 8
35378 36096  36096 3 cnt 8
32944 32876  33462 1 cnt 9
32956 32876  33462 2 cnt 9
32964 32876  33462 3 cnt 9
35682 35620  36213 1 cnt 10
35720 35620  36213 2 cnt 10
35710 35620  36213 3 cnt 10


In [185]:
timebenches = [TIMEbenchSA, TIMEbenchREDUX, TIMEbenchPD]

for t in timebenches
    println(t)
end
println()

for r in benchREDUX
    println(r[2])
end
println()

rel_error = Dict()
for b in benchSA
    println(mean(b[2]))
    println(std(b[2]))
end

62.668117421333335
0.000137637
0.74374953

0.020699827077611636
0.030642410460488913
0.017512402127786743
0.016375334824510535
0.017971246006389777
0.0045812346553949735
0.016579819990525817
0.0
0.04614238983005771
0.025421898620923703

0.009112331739734853
0.0014168585220413878
0.007864316846693196
0.0005243924238022248
0.015161476699938238
0.0003008321652529537
0.014055725844310052
0.0005439404524229481
0.009461052791723719
0.0011528559688291256
0.012832519172846694
0.00026786315128852546
0.00757934628138323
0.00046559834762235203
0.01750886524822695
0.003236380068018533
0.002969779172539737
0.0034869261850832686
0.010849556248007486
0.004110630996851152


In [187]:
t = []
for i in 1:length(benchSA)
    b = benchSA[i]
    push!(t, box(y=b, name="inst. "*string(i)))
end

plot([ a for a in t], Layout(title="Relative error in instances 40-BB with high T=1000"))

#         df = DataFrame(;[Symbol(k)=>convert(Array{Float64,1}, v) for (k,v) in solution]...)
#         for k in n_4
#             push!(x, 
#                 box(y=df[!,string(k)], name=k, marker_color=names[name_i]))#names[name_i]))
            
#         end    
#         layout = Layout(title="Set of 500 instances of ZKC dataset using "*methods[name_i], yaxis_type = "log", xaxis_title="Number of items", yaxis_title="Log time in seconds", showlegend=false)
#         p = plot([x[i] for i in 1:size(x)[1]], layout)

data: [
  "box with fields name, type, and y",
  "box with fields name, type, and y",
  "box with fields name, type, and y",
  "box with fields name, type, and y",
  "box with fields name, type, and y",
  "box with fields name, type, and y",
  "box with fields name, type, and y",
  "box with fields name, type, and y",
  "box with fields name, type, and y",
  "box with fields name, type, and y"
]

layout: "layout with fields margin, template, and title"